In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter  import PorterStemmer
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud

import pandas as pd
import random, time
from babel.dates import format_date, format_datetime, format_time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [39]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Preprocessing functions

In [11]:
def import_data():
    """
    This function imports the data set, adds labels and returns a Pandas Dataframe, without duplicates
    """

    # import small train and test datasets
    #train_neg = [tweet[:-1] for tweet in open('./twitter-datasets/train_neg.txt').readlines()]
    #train_pos = [tweet[:-1] for tweet in open('./twitter-datasets/train_pos.txt').readlines()]
    
    #Kaggle version
    train_neg = [tweet[:-1] for tweet in open('/kaggle/input/tweetsdataset/train_neg.txt').readlines()]
    train_pos = [tweet[:-1] for tweet in open('/kaggle/input/tweetsdataset/train_pos.txt').readlines()]
    
    
    X, y = train_neg + train_pos, [-1 for i in range(len(train_neg))]+[1 for i in range(len(train_pos))]
    df = pd.DataFrame(list(zip(y, X)), columns = ['label','text'], dtype = str)
    df.drop_duplicates(inplace = True)# Delete duplicate Tweets
    df['label'] = df['label'].astype(int)
    return df
    
    

In [7]:
def cleaning_data(df):

    # remove special characters from text column
    df.text = df.text.str.replace('[#,@,&]', '')
    
    #Replace special characters
    df.text = df.text.str.replace('(','')
    df.text = df.text.str.replace(')','')
    df.text = df.text.str.replace('=','')
    df.text = df.text.str.replace('!','')
    df.text = df.text.str.replace('?','')
    df.text = df.text.str.replace('"','')
    df.text = df.text.str.replace('_','')
    df.text = df.text.str.replace('-','')
    df.text = df.text.str.replace(',','')
    df.text = df.text.str.replace('.','')
    df.text = df.text.str.replace(';','')
    df.text = df.text.str.replace('+','')
    df.text = df.text.str.replace('<user>','')
    df.text = df.text.str.replace('<rt>','')
    df.text = df.text.str.replace(':','')
    df.text = df.text.str.replace('/','')
    df.text = df.text.str.replace('<','')
    df.text = df.text.str.replace('>','')
    df.text = df.text.str.replace('\'s','')
    
    # Remove digits
    df.text = df.text.str.replace('\d*','')
    
    #Remove www
    df.text = df.text.str.replace('w{3}','')
    # remove urls
    df.text = df.text.str.replace("http\S+", "")
    # remove multiple spaces with single space
    df.text = df.text.str.replace('\s+', ' ')
    #remove all single characters (except "i")
    df.text = df.text.str.replace(r'\s+[a-hA-H]\s+', '')
    df.text = df.text.str.replace(r'\s+[j-zJ-Z]\s+', '')
    df.text = df.text.str.replace(r'\s+[i-iI-I]\s+',' ')
    return df


In [36]:
def remove_stopwords(df):
    stop_words = ['i', 'me', 'my', 'myself', 'we','url' 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain']
    stop_words.extend(['u', 'wa', 'ha','ho', 'would', 'com', 'user','<user>', '<rt>' 'url', 'rt', 'custom picture', 'i\'m', 'picture frame','<url>', 'positer frame', 'x','i\'ll'])
    stop_words.remove('not')
    stop_words.remove('no')
    stop_words.remove('nor')
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return df



In [45]:
def Porter_stemmer(df):
    stemmer = PorterStemmer()
    df['text'] = df['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    return df

def snow_ball_stemmer(df):
    snow_stemmer = SnowballStemmer(language='english')
    df['text'] = df['text'].apply(lambda x: ' '.join([snow_stemmer.stem(word) for word in x.split()]))
    return df

def lemmatize_text(df):
    #lemmatize text column by using a lemmatize function
    lemmatizer = nltk.stem.WordNetLemmatizer()
    df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
    return df

# Vectorizer

In [49]:
def Basic_Vectorizer(df):
    text = df['text']
    y = df['label'].to_numpy()#.reshape(-1,1)
    
    basic_vectorizer = CountVectorizer(binary=True)
    basic_vectorizer.fit(text)
    X = basic_vectorizer.transform(text)
    
    return X, y



def N_Gram_Vectorizer(df, N):
    text = df['text']
    y = df['label'].to_numpy()#.reshape(-1,1)
    
    #adding two or three word sequences (bigrams or trigrams)
    ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, N))
    ngram_vectorizer.fit(text)
    X = ngram_vectorizer.transform(text)
    
    return X, y


def GloVe_Vectorizer(df):
    text = df['text']
    y = df['label'].to_numpy()#.reshape(-1,1)
    
    # A compléter
    
    return X,y


def Roberta_Vectorizer(df):
    text = df['text']
    y = df['label'].to_numpy()#.reshape(-1,1)
    
    # A compléter
    
    return X,y

# SVD

In [28]:
def SVD_preprocessing(X, y, N):
    clf = TruncatedSVD(100)
    Xpca = clf.fit_transform(X)
    return Xpca, y

In [32]:
def evaluate_method(X,y, message):
    random.seed(42)
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.75)
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    print ("Accuracy "+message+" : %s" % (accuracy_score(y_val, lr.predict(X_val))))
    return None

# Preprocessing methods testing

In [40]:
def test_clean_data():
    
    #Prediction accuracy with clean data
    df1 = import_data()
    df1 = cleaning_data(df1)
    X1, y1 = Basic_Vectorizer(df1)
    evaluate_method(X1, y1, "with clean data")
    
    #Prediction accuracy without clean data
    
    df2 = import_data()
    X2, y2 = Basic_Vectorizer(df2)
    evaluate_method(X2, y2, "without clean data")
    
    #Solution of test
    
    #Accuracy with clean data : 0.7924378460656063
    #Accuracy without clean data : 0.7940041031523681
    #As the Accuracy is equal, we keep this process
    
    return None

test_clean_data()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

Accuracy with clean data : 0.7923496062297324


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy without clean data : 0.790430389799475


In [37]:
def test_stopwords():
    
    #Prediction accuracy with  deleting stopwords
    df1 = import_data()
    df1 = cleaning_data(df1)
    df1 = remove_stopwords(df1)
    X1, y1 = Basic_Vectorizer(df1)
    evaluate_method(X1, y1, "with deleting stopwords")
    
    #Prediction accuracy without deleting stopwords
    
    df2 = import_data()
    df2 = cleaning_data(df2)
    X2, y2 = Basic_Vectorizer(df2)
    evaluate_method(X2, y2, "without deleting stopwords")
    
    #Solution of test
    
    #Accuracy with deleting stopwords : 0.7828858838322561
    #Accuracy without deleting stopwords : 0.790011250579074
    #As the Accuracy is equal, we should not keep this process
    
    
    return None

test_stopwords()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

Accuracy with deleting stopwords : 0.7828858838322561


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

Accuracy without deleting stopwords : 0.790011250579074


In [47]:
def test_normalization():
    
    #Prediction accuracy without  deleting normalization
    df1 = import_data()
    df1 = cleaning_data(df1)
    df1 = remove_stopwords(df1)#voir si on garde
    X1, y1 = Basic_Vectorizer(df1)
    evaluate_method(X1, y1, "without normalization")
    
    #Prediction accuracy with Porter
    df2 = import_data()
    df2 = cleaning_data(df2)
    df2 = remove_stopwords(df2) #voir si on garde
    df2 = Porter_stemmer(df2)
    X2, y2 = Basic_Vectorizer(df2)
    evaluate_method(X2, y2, "with Porter Stemmer")
    
    
    #Prediction accuracy with SnowBall Stemmer
    df3 = import_data()
    df3 = cleaning_data(df3)
    df3 = remove_stopwords(df3) #voir si on garde
    df3 = snow_ball_stemmer(df3)
    X3, y3 = Basic_Vectorizer(df3)
    evaluate_method(X3, y3, "with Snowball Stemmer")   
    
    
    #Prediction accuracy with Lemmatizer
    df4 = import_data()
    df4 = cleaning_data(df4)
    df4 = remove_stopwords(df4) #voir si on garde
    df4 = lemmatize_text(df4)
    
    X4, y4 = Basic_Vectorizer(df4)
    evaluate_method(X4, y4, "with lemmatization")   
    
    
    #Solution of test
    
    #Accuracy without normalization : 0.782091725309391
    #Accuracy with Porter Stemmer : 0.7836359224371843
    #Accuracy with Snowball Stemmer : 0.7831285433809093
    #Accuracy with lemmatization : XXX
    #As the Accuracy is nearly equal, we keep method Porter Stemmer, that is the best for reproductibility of results and is short to implement
    
    return None

test_normalization()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

vs celtics regular season fucked play playoffs


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy without normalization : 0.7848492201804504


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

...

vs celtic regular season fuck play playoff


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy with Snowball Stemmer : 0.7800622090842911


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - '/root/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/share/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [50]:
def test_vectorization():
    
    #Prediction accuracy with basic vectorization
    df1 = import_data()
    df1 = cleaning_data(df1)
    df1 = remove_stopwords(df1)#voir si on garde
    df1 = Porter_stemmer(df1)
    X1, y1 = Basic_Vectorizer(df1)
    evaluate_method(X1, y1, "with basic vectorization")
    
    
    #Prediction accuracy with 2-Grams vectorization
    df1 = import_data()
    df1 = cleaning_data(df1)
    df1 = remove_stopwords(df1)#voir si on garde
    df1 = Porter_stemmer(df1)
    X1, y1 = N_Gram_Vectorizer(df1,2)
    evaluate_method(X1, y1, "with 2-grams vectorization")
    
    
    
    #Prediction accuracy with 3-Grams vectorization
    df1 = import_data()
    df1 = cleaning_data(df1)
    df1 = remove_stopwords(df1)#voir si on garde
    df1 = Porter_stemmer(df1)
    X1, y1 = N_Gram_Vectorizer(df1,3)
    evaluate_method(X1, y1, "with 3-grams vectorization")
    
    
    #Prediction accuracy with Glove vectorization
    
    
    
    #Prediction accuracy with Roberta vectorization
    
    
    #Solution of test
    
    #Accuracy with basic vectorization : 0.7799739692484172
    #Accuracy with 2-grams vectorization : 0.7945776620855485
    #Accuracy with 3-grams vectorization : 0.7926143257373541
    

    return None

test_vectorization()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

Accuracy with basic vectorization : 0.7822461450221703


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

Accuracy with 2-grams vectorization : 0.7945776620855485


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when

Accuracy with 3-grams vectorization : 0.7926143257373541


## Predictions

In [ ]:
#Preparing data
test_data = [tweet[:-1] for tweet in open('./twitter-datasets/test_data.txt').readlines()]
df_test = pd.DataFrame(list(test_data), columns = ['text'], dtype = str)
df_test =cleaning_data(df_test)
df_test['text'] = df_test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
X_test = ngram_vectorizer.transform(df_test['text'])